In [118]:
import numpy as np
import pandas as pd

# Apply Environmental Flow Requirements

Applies the selected Environmental Assurance Categories for each channel and determines the required EWR flows.

In [119]:
category_sets=pd.read_csv("ewr_section_categories.csv")
category_sets

,Section,Recommendation 1,All Cs,All Bs
0,A,B,C,B
1,C,B,C,B
2,D,B,C,B
3,E,B,C,B
4,FG,C,C,B


## Selected Assurance Category Set ->

In [120]:
category_set_name='Recommendation 1'

In [121]:
category_set=category_sets.loc[:,['Section',category_set_name]]
category_set=category_set.rename(columns={category_set_name:'Category'})
category_set

,Section,Category
0,A,B
1,C,B
2,D,B
3,E,B
4,FG,C


## Functions

In [151]:
def flowBandNo(exceed):
    if exceed <= 0.1:
        return 1
    elif  exceed <= 0.4:
        return 2
    elif  exceed <= 0.7:
        return 3
    elif  exceed <= 0.9:
        return 4
    else:
        return 5
    
def flowBandLabel(bandNo):
    if bandNo == 1:
        return 'Wet'
    elif  bandNo == 2:
        return 'Mod_Wet'
    elif  bandNo == 3:
        return 'Normal'
    elif  bandNo ==4:
        return 'Mod_Dry'
    else:
        return 'Dry'    
    
def ewrLookup(section, band, month):
    return ewrs.loc[(section,month)][band]

## Load various data files

Load the daily time series which includes the exceedance values used for calculating daily EWRs.

Load the EWR Flow Sets which specify the required EWRs by assurance category, channel section and calendar month

In [129]:
daily=pd.read_csv("prepared_daily_ewrx.csv")
daily=daily.set_index(pd.to_datetime(daily['Date'],format="%Y-%m-%d"))
daily=daily.drop('Date',axis=1)

daily

,LaggedDate,VicFalls,Conversion,Flow,Exceedance,Year,Month,Day,MonthId,WaterYear,WaterMonth,WaterDay,Volume,EWRRefExceedance
Date,,,,,,,,,,,,,,
1924-10-01,1924-10-12,100.0,1.111331,111.133124,0.999,1924,10,1,1924.10,1924,1,1,0.009602,0.50
1924-10-02,1924-10-13,100.0,1.111331,111.133124,0.999,1924,10,2,1924.10,1924,1,2,0.009602,0.50
1924-10-03,1924-10-14,100.0,1.111331,111.133124,0.999,1924,10,3,1924.10,1924,1,3,0.009602,0.50
1924-10-04,1924-10-15,100.0,1.111331,111.133124,0.999,1924,10,4,1924.10,1924,1,4,0.009602,0.50
1924-10-05,1924-10-16,100.0,1.111331,111.133124,0.999,1924,10,5,1924.10,1924,1,5,0.009602,0.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-09-26,2017-10-07,204.0,0.995184,203.017496,0.950,2017,9,26,2017.09,2016,12,361,0.017541,0.82
2017-09-27,2017-10-08,204.0,0.995184,203.017496,0.950,2017,9,27,2017.09,2016,12,362,0.017541,0.82
2017-09-28,2017-10-09,204.0,0.995184,203.017496,0.950,2017,9,28,2017.09,2016,12,363,0.017541,0.82


In [130]:
ewr_flow_sets=pd.read_csv("ewr_flow_sets.csv")
ewr_flow_sets=ewr_flow_sets.set_index(['Section','Category'])
ewr_flow_sets

Month  Wet  Mod_Wet  Normal  Mod_Dry  Dry
Section Category                                           
A       A            10   75       70      60       47   38
        A            11   73       69      58       45   37
        A            12   77       72      62       48   40
        A             1   83       77      66       52   46
        A             2   90       81      69       56   51
...                 ...  ...      ...     ...      ...  ...
FG      D             5   76       76      66       49   42
        D             6   70       57      46       39   31
        D             7   64       53      41       31   27
        D             8   51       43      32       25   22
        D             9   36       31      23       17   15

[240 rows x 6 columns]

## Flow Sets
Select the EWR Flow Set for each channel section according to the Category Set chosen above (eg *Recommendation 1*)

In [131]:
ewrs=category_set.join(ewr_flow_sets,on=['Section','Category']).set_index(['Section','Month'])
ewrs_A=ewrs.loc['A']
ewrs_C=ewrs.loc['C']
ewrs_D=ewrs.loc['D']
ewrs_E=ewrs.loc['E']
ewrs_FG=ewrs.loc['FG']

ewrs_A


,Category,Wet,Mod_Wet,Normal,Mod_Dry,Dry
Month,,,,,,
10,B,44,41,37,31,24
11,B,42,40,36,29,23
12,B,44,42,38,31,26
1,B,49,46,41,34,29
2,B,55,49,43,36,31
3,B,68,59,49,41,36
4,B,84,70,54,45,40
5,B,71,65,55,47,42
6,B,67,62,53,45,39


## Flow Bands
Annotate the daily flow series with the EWR Flow Band (Wet, Very Wet, Dry etc) - a lookup against the EWR Exceedance

In [132]:

daily['EWRBandNo']=daily.apply(lambda x: flowBandNo(x['EWRRefExceedance']),axis=1) 
daily['EWRBandLabel']=daily.apply(lambda x: flowBandLabel(x['EWRBandNo']),axis=1) 
daily

,LaggedDate,VicFalls,Conversion,Flow,Exceedance,Year,Month,Day,MonthId,WaterYear,WaterMonth,WaterDay,Volume,EWRRefExceedance,EWRBandNo,EWRBandLabel
Date,,,,,,,,,,,,,,,,
1924-10-01,1924-10-12,100.0,1.111331,111.133124,0.999,1924,10,1,1924.10,1924,1,1,0.009602,0.50,3,Normal
1924-10-02,1924-10-13,100.0,1.111331,111.133124,0.999,1924,10,2,1924.10,1924,1,2,0.009602,0.50,3,Normal
1924-10-03,1924-10-14,100.0,1.111331,111.133124,0.999,1924,10,3,1924.10,1924,1,3,0.009602,0.50,3,Normal
1924-10-04,1924-10-15,100.0,1.111331,111.133124,0.999,1924,10,4,1924.10,1924,1,4,0.009602,0.50,3,Normal
1924-10-05,1924-10-16,100.0,1.111331,111.133124,0.999,1924,10,5,1924.10,1924,1,5,0.009602,0.50,3,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-09-26,2017-10-07,204.0,0.995184,203.017496,0.950,2017,9,26,2017.09,2016,12,361,0.017541,0.82,4,Mod_Dry
2017-09-27,2017-10-08,204.0,0.995184,203.017496,0.950,2017,9,27,2017.09,2016,12,362,0.017541,0.82,4,Mod_Dry
2017-09-28,2017-10-09,204.0,0.995184,203.017496,0.950,2017,9,28,2017.09,2016,12,363,0.017541,0.82,4,Mod_Dry


## EWR Flows
Calculate the required flows by channel

In [141]:
tmp=daily.join(ewrs_A,on='Month')
daily['EWR_Channel_A']=tmp.apply(lambda x: x[x['EWRBandLabel']],axis=1)

In [142]:
tmp=daily.join(ewrs_C,on='Month')
daily['EWR_Channel_C']=tmp.apply(lambda x: x[x['EWRBandLabel']],axis=1)

In [143]:
tmp=daily.join(ewrs_D,on='Month')
daily['EWR_Channel_D']=tmp.apply(lambda x: x[x['EWRBandLabel']],axis=1)

In [144]:
tmp=daily.join(ewrs_E,on='Month')
daily['EWR_Channel_E']=tmp.apply(lambda x: x[x['EWRBandLabel']],axis=1)

In [145]:
tmp=daily.join(ewrs_FG,on='Month')
daily['EWR_Channel_FG']=tmp.apply(lambda x: x[x['EWRBandLabel']],axis=1)

In [149]:
daily['EWR_Total']=daily['EWR_Channel_A']+daily['EWR_Channel_C']+daily['EWR_Channel_D']+daily['EWR_Channel_E']+daily['EWR_Channel_FG']
daily['EWR_Proportion']=daily['EWR_Total']/daily['Flow']
daily

,LaggedDate,VicFalls,Conversion,Flow,Exceedance,Year,Month,Day,MonthId,WaterYear,...,EWRRefExceedance,EWRBandNo,EWRBandLabel,EWR_Channel_A,EWR_Channel_C,EWR_Channel_D,EWR_Channel_E,EWR_Channel_FG,EWR_Total,EWR_Proportion
Date,,,,,,,,,,,,,,,,,,,,,
1924-10-01,1924-10-12,100.0,1.111331,111.133124,0.999,1924,10,1,1924.10,1924,...,0.50,3,Normal,37,5,3,2,50,97,0.872827
1924-10-02,1924-10-13,100.0,1.111331,111.133124,0.999,1924,10,2,1924.10,1924,...,0.50,3,Normal,37,5,3,2,50,97,0.872827
1924-10-03,1924-10-14,100.0,1.111331,111.133124,0.999,1924,10,3,1924.10,1924,...,0.50,3,Normal,37,5,3,2,50,97,0.872827
1924-10-04,1924-10-15,100.0,1.111331,111.133124,0.999,1924,10,4,1924.10,1924,...,0.50,3,Normal,37,5,3,2,50,97,0.872827
1924-10-05,1924-10-16,100.0,1.111331,111.133124,0.999,1924,10,5,1924.10,1924,...,0.50,3,Normal,37,5,3,2,50,97,0.872827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-09-26,2017-10-07,204.0,0.995184,203.017496,0.950,2017,9,26,2017.09,2016,...,0.82,4,Mod_Dry,34,6,4,3,44,91,0.448237
2017-09-27,2017-10-08,204.0,0.995184,203.017496,0.950,2017,9,27,2017.09,2016,...,0.82,4,Mod_Dry,34,6,4,3,44,91,0.448237
2017-09-28,2017-10-09,204.0,0.995184,203.017496,0.950,2017,9,28,2017.09,2016,...,0.82,4,Mod_Dry,34,6,4,3,44,91,0.448237


In [152]:
daily.to_csv('prepared_daily_ewr.csv')